In [4]:
import pandas as pd
import numpy as np
import funcs, gc, sqlite3

In [22]:
query = '''
SELECT    
    "RAT.NUM_ATIVIDADE",
    "MUNICIPIO",
    "LOGRADOURO",
    "DES_ENDERECO",
    "COMPLEMENTO_ENDERECO",
    "NOME_BAIRRO",
    "LOGRADOURO2",
    "DES_ENDERECO2",
    "SETOR"
FROM
    tbl_rat
WHERE
    "SETOR" == "other"
AND "MUNICIPIO" != "ITAUNA"
AND "NAT.CODIGO" IN ('Y07001', 'Y07003', 'Y07004', 'Y07005', 'Y04012')
'''

rats_unclassif = pd.read_sql(query, 'sqlite:///gdo.db')
rats_unclassif

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR


In [23]:
df_classif = funcs.get_df_classif()
rats_unclassif.loc[:,'SETOR'] = rats_unclassif.apply(lambda row: funcs.classifica_setor(row, df_classif), axis=1)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [3]:
new_classif_dict = {rats[0]:rats[-1] for rats in rats_unclassif.values}
new_classif_dict

{'2017-039868355-001': 'other',
 '2018-009490383-001': 'other',
 '2018-010077065-001': 'other',
 '2018-012028223-001': 'other',
 '2018-012263398-001': 'other',
 '2018-016891050-001': 'other',
 '2018-021635748-001': 'other',
 '2018-023079995-001': 'other',
 '2018-025665611-001': 'other',
 '2018-026453048-001': 'other',
 '2018-026813879-001': 'other',
 '2018-028152218-001': 'other',
 '2018-028769709-001': 'other',
 '2018-030505040-001': 'other',
 '2018-033587860-001': 'other',
 '2018-033622770-001': 'other',
 '2018-039000490-001': 'other',
 '2018-044525142-001': 'other',
 '2018-048994558-001': 'other',
 '2018-053299244-001': 'other',
 '2018-055016671-001': 'other',
 '2019-002158845-001': 'other',
 '2019-005463128-001': 'other',
 '2019-010054700-001': 'other',
 '2019-014271514-001': 'other',
 '2019-014462284-001': 'other',
 '2019-014462817-001': 'other',
 '2019-014895800-001': 'other',
 '2019-015245147-001': 'other',
 '2019-015261347-001': 'other',
 '2019-015325049-001': 'other',
 '2019-0

In [17]:
query_update = '''
UPDATE tbl_rat
SET "SETOR" = ?
WHERE "RAT.NUM_ATIVIDADE" = ?
'''

with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    for nrat, setor in new_classif_dict.items():
        print((setor, nrat))
        cursor.execute(query_update, (setor, nrat))
        conn.commit()
conn.close()

In [7]:
rats = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')

In [12]:
rats[
    (rats['SETOR'] == 'other')
    & (rats['NAT.CODIGO'].str.contains('Y0700[1345]'))
    & (rats['MUNICIPIO'] != 'ITAUNA')
][[
    "RAT.NUM_ATIVIDADE",
    "MUNICIPIO",
    "LOGRADOURO",
    "DES_ENDERECO",
    "COMPLEMENTO_ENDERECO",
    "NOME_BAIRRO",
    "LOGRADOURO2",
    "DES_ENDERECO2",
    "SETOR"
]]

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR


In [20]:
rats[
    rats['ANO'] > 2018
].groupby(['CIA', 'SETOR', 'SETOR']).count()

RAT.NUM_ATIVIDADE  \
CIA     SETOR                               SETOR                                                    
139 CIA CLAUDIO                             CLAUDIO                                           3602   
        PLANALTO                            PLANALTO                                          3142   
        SAO JOSE                            SAO JOSE                                          3966   
142 CIA CARMO DO CAJURU/SAO GONCALO DO PARA CARMO DO CAJURU/SAO GONCALO DO PARA               6450   
        NITEROI                             NITEROI                                           4392   
        PORTO VELHO                         PORTO VELHO                                       3951   
51 CIA  CENTRO/PADRE EUSTAQUIO              CENTRO/PADRE EUSTAQUIO                            4846   
        LOURDES/ITATIAIUCU                  LOURDES/ITATIAIUCU                                2630   
        SANTANENSE/MORADA NOVA              SANTANENSE/MORADA NOVA                            4223   
        other                               other                                              117   
53 CIA  ALTO GOIAS                          ALTO GOIAS                                         903   
        BOM PASTOR                          BOM PASTOR                                        4632   
        HIPER CENTRO                        HIPER CENTRO                                      3516   
other   other                               other                                              154   

                                                                                 NAT.CODIGO  \
CIA     SETOR                               SETOR                                             
139 CIA CLAUDIO                             CLAUDIO                                    3602   
        PLANALTO                            PLANALTO                                   3142   
        SAO JOSE                            SAO JOSE                                   3966   
142 CIA CARMO DO CAJURU/SAO GONCALO DO PARA CARMO DO CAJURU/SAO GONCALO DO PARA        6450   
        NITEROI                             NITEROI                                    4392   
        PORTO VELHO                         PORTO VELHO                                3951   
51 CIA  CENTRO/PADRE EUSTAQUIO              CENTRO/PADRE EUSTAQUIO                     4846   
        LOURDES/ITATIAIUCU                  LOURDES/ITATIAIUCU                         2630   
        SANTANENSE/MORADA NOVA              SANTANENSE/MORADA NOVA                     4223   
        other                               other                                       117   
53 CIA  ALTO GOIAS                          ALTO GOIAS                                  903   
        BOM PASTOR                          BOM PASTOR                                 4632   
        HIPER CENTRO                        HIPER CENTRO                               3516   
other   other                               other                                       154   

                                                                                 NAT.DESCRICAO  \
CIA     SETOR                               SETOR                                                
139 CIA CLAUDIO                             CLAUDIO                                       3602   
        PLANALTO                            PLANALTO                                      3142   
        SAO JOSE                            SAO JOSE                                      3966   
142 CIA CARMO DO CAJURU/SAO GONCALO DO PARA CARMO DO CAJURU/SAO GONCALO DO PARA           6450   
        NITEROI                             NITEROI                                       4392   
        PORTO VELHO                         PORTO VELHO                                   3951   
51 CIA  CENTRO/PADRE EUSTAQUIO              CENTRO/PADRE EUSTAQUIO                        4846   
        LOURDES/ITATIAIUCU                  L